In [2]:
import torch
import os
import sys

import pickle
import itertools
from tqdm import tqdm

from contextlib import redirect_stdout, redirect_stderr

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, "../src/")))
from engine import run_vit_experiment, run_cnn_experiment, test_step
from config import VIT_MODEL_CONFIG_MAP, CNN_MODEL_CONFIG_MAP, OPTIMIZER_CONFIG_MAP
from utils import _DevNull
from data_loader import create_dataloaders

print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

/home/hoai-linh.dao/Envs/brats-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
Number of GPUs: 2
GPU 0: NVIDIA L40
GPU 1: NVIDIA L40


In [ ]:
import torch
from torchvision import transforms
from model_utils import load_vit_model
from torch import nn

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_cfg = {
    "model_name": "torchvision.ViT_L_16_Weights.IMAGENET1K_V1",
    "pretrained": True,
    "num_classes": 4
}
checkpoint_path = "/home/hoai-linh.dao/Works/BraTS/results/self-experiments/extended-dataset-MRI-scan/weights/adam/L16_1K_case3.pth"

model, model_transforms = load_vit_model(device, model_cfg["model_name"], model_cfg['num_classes'])

model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.to(device)
model.eval()

data_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    model_transforms,
])

_, _, test_loader = create_dataloaders(
    dataset_dir="/home/hoai-linh.dao/Works/BraTS/dts/MRI-Scan",
    transform=data_transforms,
    batch_size=32,
    num_workers=4,
    train_ratio=0.7,
    val_ratio=0.1,
    test_ratio=0.2,
    seed=42,
    mode="split_folder"
)

loss_fn = nn.CrossEntropyLoss()

test_loss, test_acc = test_step(model=model,
                               dataloader=test_loader,
                               loss_fn=loss_fn,
                               device=device)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Test Loss: 0.0305, Test Accuracy: 0.9954
